<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

In [1]:
%tensorflow_version 1.8
import tensorflow as tf
print(tf.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.8`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
name = [ ]
name_List = [ ]
abstract = [ ]
abstract_List=[ ]
page_number = [0,10,20,30,40,50,60,70,80,90]
for i in page_number:
 page = requests.get('https://citeseerx.ist.psu.edu/search;jsessionid=87FF6C66EA09F22314C131669600CF98?q=natural+language+processing&t=doc&sort=rlv&start='+ str (i))
 soup = BeautifulSoup(page.content,'html.parser')
 name.append(soup.find_all('a',class_='remove doc_details'))
 abstract.append(soup.find_all('div',class_='pubabstract'))

In [3]:
for j in name:
  for k in range(len(j)):
    name_List.append(j[k].get_text())

In [4]:
for l in abstract:
  for m in range(len(l)):
    abstract_List.append(l[m].get_text())

In [5]:
df = pd.DataFrame({'Article':name_List,'Abstract': abstract_List})

In [6]:
df.head()

,Article,Abstract
0,\n Foundations of statistical...,\n Abstract not found\n ...
1,\n A Maximum Entropy approach...,\n describe a method for s...
2,\nNatural Language Processing\n,\n Scaling conditional rand...
3,\n Linguistics and Natural La...,\n The paper addresses the ...
4,\nNatural Language Processing\n,\n In most natural language...


In [7]:
#1 Noise removal
df['Article'] = df['Article'].replace('\n','',regex = True)
df['Abstract'] = df['Abstract'].replace('\n','',regex = True)
df['Article'] = df['Article'].replace('[^\w\s]','',regex = True)
df['Abstract'] = df['Abstract'].replace('[^\w\s]','',regex = True)
#2 Remove numbers
df['Article'] = df['Article'].str.replace('\d+','')
df['Abstract'] = df['Abstract'].str.replace('\d+','')
#3 Remove StopWords
stop_Words = ["i", "me", "my", "myself", "we", "our", "ours", 
              "ourselves", "you", "your", "yours", "yourself",
              "yourselves", "he", "him", "his", "himself", "she", "her",
              "hers", "herself", "it", "its", "itself", "they", "them",
              "their", "theirs", "themselves", "what", "which", "who",
              "whom", "this", "that", "these", "those", "am", "is",
              "are", "was", "were", "be", "been", "being", "have",
              "has", "had", "having", "do", "does", "did", "doing",
              "a", "an", "the", "and", "but", "if", "or", "because",
              "as", "until", "while", "of", "at", "by", "for", "with",
              "about", "against", "between", "into", "through",
              "during", "before", "after", "above", "below", "to",
              "from", "up", "down", "in", "out", "on", "off", "over",
              "under", "again", "further", "then", "once", "here",
              "there", "when", "where", "why", "how", "all", "any",
              "both", "each", "few", "more", "most", "other", "some",
              "such", "no", "nor", "not", "only", "own", "same", "so",
              "than", "too", "very", "s", "t", "can", "will", "just",
              "don", "should", "now"]
# stop = stopwords.words('english')
df['Article'] = df['Article'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_Words))
df['Abstract'] = df['Abstract'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_Words))
#4 Lowercase all text
df['Article'] = df['Article'].str.lower()
df['Abstract'] = df['Abstract'].str.lower()
#5 Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
df['Article'] = df['Article'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
df['Abstract']= df['Abstract'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
#6 Lemmatization
from textblob import Word
import nltk
nltk.download('wordnet')
df['Abstract']=df['Abstract'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [8]:
df.head(10)

,Article,Abstract
0,foundat statist natur languag process,abstract found
1,a maximum entropi approach natur languag process,describ method statist model base maximum entr...
2,natur languag process,scale condit random field natur languag proces...
3,linguist natur languag process,the paper address issu cooper linguist natur l...
4,natur languag process,in natur languag process applic descript logic...
5,natur languag process almost scratch,we propos unifi neural network architectur lea...
6,natur languag process,natur languag process the subject natur langua...
7,natur languag processingrobot,robot interact human facetofac use natur langu...
8,tutori natur languag process,natur languag languag spoken human current yet...
9,ambigu natur languag process,abstract ambigu refer abil one mean understood...


# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [12]:
# Write your code here
import nltk

In [13]:
from nltk.util import ngrams

In [14]:
df['Abstract'] = df['Abstract'].apply(lambda x: x.split())

In [16]:
df['Abstract'].head(3)

0                                    [abstract, found]
1    [describ, method, statist, model, base, maximu...
2    [scale, condit, random, field, natur, languag,...
Name: Abstract, dtype: object

In [18]:
df['TriGram_Abstract'] = ngrams(df['Abstract'], 3))

SyntaxError: ignored

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [10]:
# Write your code here





# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [11]:
# The GitHub link of your final csv file

# Link: 